# Using Tools with Language Models

**Tool calling** allows language models to interact with external functions and APIs, extending their capabilities beyond text generation. This notebook demonstrates how to implement and use tools with LLMs.

## What You Will Learn

1. Setting up tool declarations for LLMs
2. Implementing external API integrations
3. Creating currency conversion tools
4. Testing tools independently
5. Combining tools with structured output
6. Multi-step reasoning with function calls

---

## 🚀 Section 1: Setup and Installation

In [1]:
%pip install -U -q json-repair

---

## 🔌 Section 2: Connecting to the Model

### 📡 Google GenAI Client Setup

We will use Google's Gemini model with tool calling capabilities.

In [2]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY_1')

In [3]:
import os
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from datetime import date, datetime
from enum import Enum
from dotenv import load_dotenv
from json_repair import repair_json

load_dotenv()

MODEL_NAME = "gemini-2.5-flash-lite"
client = genai.Client(
    api_key=api_key
)

response = client.models.generate_content(
    model=MODEL_NAME,
    contents="Explain how AI works in 5 words"
)

print(response.text)

Learning from data, predicting outcomes.


---

## 📊 Section 3: Defining Data Models

### 💱 Currency Conversion Schema

Define Pydantic models for currency conversion and invoice data.

In [4]:
class CurrencyConversion(BaseModel):
    original_total: float = Field(description="Original amount before conversion")
    original_currency: str = Field(description="Original currency code")
    converted_total: float = Field(description="Converted amount")
    converted_currency: str = Field(description="Target currency code")

---

## 💻 Section 3: Tool Implementation

### ⚙️ Implementing the Currency Conversion Function

The actual implementation that will be called when the LLM requests currency conversion.

In [5]:
import requests

def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict[str, float | str]:
    url = f"https://api.frankfurter.dev/v1/latest?from={from_currency}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if to_currency not in data['rates']:
            available_currencies = ', '.join(data['rates'].keys())
            raise Exception(f"Currency '{to_currency}' not found in available rates. Available: {available_currencies}")

        rate = data['rates'][to_currency]
        converted_amount = amount * rate

        return {
            "original_total": amount,
            "original_currency": from_currency,
            "converted_total": converted_amount,
            "converted_currency": to_currency,
        }
    else:
        raise Exception(f"Failed to fetch conversion rates: {response.status_code}")

---

## 🔧 Section 4: Tool Declaration

### 📋 Declaring the Currency Conversion Tool

Tool declarations define the function signature that the LLM can understand and call.

In [6]:
convert_currency_declaration = types.FunctionDeclaration(
    name="convert_currency",
    description="Converts an amount from one currency to another using the Frankfurter API.",
    parameters={
        "type": "object",
        "properties": {
            "amount": {
                "type": "number",
                "description": "The amount to be converted."
            },
            "from_currency": {
                "type": "string",
                "description": "The currency code of the original currency (e.g., 'USD')."
            },
            "to_currency": {
                "type": "string",
                "description": "The currency code to convert to (e.g., 'EUR')."
            }
        },
        "required": ["amount", "from_currency", "to_currency"]
    }
)

---

## 🧪 Section 5: Testing the Tool

### 🔬 Testing Currency Conversion Independently

Test the tool function directly before using it with the LLM.

In [7]:
result = convert_currency(amount=100, from_currency="USD", to_currency="PLN")
conversion = CurrencyConversion.model_validate(result)
print(conversion.model_dump_json(indent=2))

{
  "original_total": 100.0,
  "original_currency": "USD",
  "converted_total": 365.21,
  "converted_currency": "PLN"
}


---

## 🎯 Section 6: Use currency tool
Using tool with LLM

In [10]:
def use_currency_tool_with_llm(
    prompt: str,
    client: genai.Client
) -> str:
    """Simple example of using a tool with LLM"""
    system_prompt = """You are a helpful assistant that can convert currencies.
    When asked about currency conversion, use the convert_currency tool."""

    tools = types.Tool(function_declarations=[convert_currency_declaration])

    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=contents,
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            tools=[tools],
            temperature=0.01,
        )
    )

    # Check if the model wants to call a function
    if response.candidates and response.candidates[0].content.parts:
        part = response.candidates[0].content.parts[0]

        if part.function_call:
            function_call = part.function_call

            if function_call.name == "convert_currency":
                # Execute the function
                conversion_result = convert_currency(**function_call.args)

                # Create function response
                function_response_part = types.Part.from_function_response(
                    name=function_call.name,
                    response=conversion_result,
                )

                # Add the function response to the conversation
                contents.append(response.candidates[0].content)
                contents.append(types.Content(role="user", parts=[function_response_part]))

                # Get final response from the model
                final_response = client.models.generate_content(
                    model=MODEL_NAME,
                    contents=contents,
                    config=types.GenerateContentConfig(
                        system_instruction=system_prompt,
                        temperature=0.01,
                    )
                )
                return final_response.text

    return response.text

In [9]:
test_invoice_text = """
INVOICE

Invoice Number: INV-2025-100
Date: January 15, 2025

From:
Tech Solutions Inc.
123 Business Street
San Francisco, CA 94105

To:
Acme Corporation
456 Market Street
New York, NY 10001

Items:
1. Web Development - 40 hours @ $150/hr = $6,000
2. Cloud Hosting - 1 month @ $500 = $500

Total: $6,500 USD
Payment Terms: Net 30 days
"""

test_currentcy_exchange_prompt = "\n\nPlease convert total amount into PLN"

respose = use_currency_tool_with_llm(
    test_invoice_text + test_currentcy_exchange_prompt,
    client,
)

print(f"Response: {respose}")

Response: The total amount of $6,500 USD is equivalent to 23,738.65 PLN.


---

## 📚 Summary

### ✨ Key Concepts Covered

1. **Tool Declarations**: Defining function signatures for LLM integration
2. **External APIs**: Integrating with third-party services (Frankfurter API)
3. **Tool Implementation**: Building the actual tool function
4. **Tool Calling Flow**: Understanding multi-step LLM reasoning
5. **Function Responses**: Processing tool outputs
6. **Structured Output + Tools**: Combining multiple capabilities

### 💡 Best Practices

- ✅ **Define clear tool signatures** with detailed descriptions
- ✅ **Validate tool inputs** before execution
- ✅ **Handle API errors** gracefully
- ✅ **Test tools independently** before LLM integration
- ✅ **Process tool responses** before returning to LLM
- ✅ **Combine tools strategically** with structured output

### 🎯 Next Steps

- 🔹 Implement additional tools (geocoding, validation, etc.)
- 🔹 Build multi-tool workflows
- 🔹 Add error recovery mechanisms
- 🔹 Create tool chaining pipelines

---

### 🎓 Congratulations!

You now understand how to integrate external tools with language models, enabling them to perform actions beyond text generation and interact with real-world APIs and services.